In [1]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

In [2]:
data = pd.read_csv("merged_final.csv")

C:\Users\alish\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (6,13,29,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# select only the user_id, rating, book_id and title
df = data[['user_id','rating', 'book_id', 'title', 'read_at']]

In [ ]:
df.shape

In [4]:
df = df.dropna(subset = ['read_at']).reset_index()

In [5]:
df = df[['user_id','rating', 'book_id', 'title']]
df.head()

,user_id,rating,book_id,title
0,d1e368a7d2870eb6fbf6e0d350568a2d,4,23310161,The Day the Crayons Came Home
1,bda576847fc8af9494d83d73bb8fea96,5,23310161,The Day the Crayons Came Home
2,528f8e3516cd2f4b8184a5b6801d66a5,4,23310161,The Day the Crayons Came Home
3,1a38ade7d5888c6c22446ba79956dea6,5,23310161,The Day the Crayons Came Home
4,8bf745a1e2b3ec721ad079990111f114,5,23310161,The Day the Crayons Came Home


In [6]:
# Select only those users having more than 20 
num = pd.DataFrame(df.groupby('user_id')['rating'].count())
num = num[num['rating']>=50]
num.reset_index()

df2 = pd.merge(num,df,on='user_id').drop_duplicates()
df2.head()

,user_id,rating_x,rating_y,book_id,title
0,004f6d189c4546a39db2162986c0d9e1,389,4,25205336,Playtime: A Mutts Treasury
2,004f6d189c4546a39db2162986c0d9e1,389,5,15704307,"Saga, Vol. 1 (Saga, #1)"
3,004f6d189c4546a39db2162986c0d9e1,389,4,31338724,"Paper Girls, Vol. 2 (Paper Girls, #2)"
4,004f6d189c4546a39db2162986c0d9e1,389,4,21823465,"Alex + Ada, Vol. 1"
5,004f6d189c4546a39db2162986c0d9e1,389,4,32273168,"The Flintstones, Vol. 1"


In [7]:
# map the user and book IDs to numbers
df2["user_id_int"] = df2['user_id'].rank(method ='dense').astype(int)
df2["book_id_int"] = df2['book_id'].rank(method ='dense').astype(int) 
df1 = df2[['user_id_int', 'book_id_int','rating_y', 'title']]
df1.head()

,user_id_int,book_id_int,rating_y,title
0,1,29573,4,Playtime: A Mutts Treasury
2,1,18451,5,"Saga, Vol. 1 (Saga, #1)"
3,1,34387,4,"Paper Girls, Vol. 2 (Paper Girls, #2)"
4,1,25785,4,"Alex + Ada, Vol. 1"
5,1,34793,4,"The Flintstones, Vol. 1"


In [8]:
df1 = df1.rename(columns={"user_id_int": "user_id", "book_id_int": "book_id", "rating_y": "rating", "title":"title"})

In [9]:
books_df = df1[['book_id', 'title']].drop_duplicates()
ratings_df = df1[['user_id','rating', 'book_id']].drop_duplicates()
df1.to_csv('findbook.csv')

In [10]:
R_df = ratings_df.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
R_df.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,36373,36374,36375,36376,36377,36378,36379,36380,36381,36382
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

C:\Users\alish\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [12]:
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)

In [13]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [14]:
preds_df.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,36373,36374,36375,36376,36377,36378,36379,36380,36381,36382
0,-0.105195,-0.011432,-0.046829,0.298328,0.098755,0.104956,-0.029763,0.015393,-0.012704,-0.001990,...,-0.000338,-0.000780,-0.000338,0.030572,-0.030451,-0.000338,-0.054847,-0.007945,-0.007945,-0.006759
1,0.008250,0.000460,0.011664,-0.043694,-0.016482,-0.024391,0.038526,0.008252,-0.000775,0.001219,...,0.001680,0.001058,0.001680,0.004356,0.003975,0.001680,-0.011466,-0.000628,-0.000628,0.001316
2,-0.040625,0.007176,0.011748,0.154810,0.019599,0.101899,0.016669,0.014191,0.000270,0.012741,...,0.007299,0.031895,0.007299,-0.018604,0.033475,0.007299,-0.024068,0.005249,0.005249,0.006664
3,0.006676,0.001351,-0.001096,-0.061567,-0.056992,-0.051097,0.023871,0.006387,-0.000503,0.003100,...,0.003874,0.005351,0.003874,0.047762,-0.000595,0.003874,-0.022174,0.006246,0.006246,0.004639
4,0.015510,-0.001496,0.009388,-0.030741,-0.000991,-0.028768,0.012946,0.002266,-0.002218,-0.001511,...,-0.001167,-0.005476,-0.001167,-0.007768,-0.002506,-0.001167,0.052371,-0.000988,-0.000988,-0.002051


In [27]:
def recommend_books(predictions_df, userID, books_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the book information.
    user_data = original_ratings_df[original_ratings_df.user_id == (userID)]
    user_full = (user_data.merge(books_df, how = 'left', left_on = 'book_id', right_on = 'book_id').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} books. The first 5 are:'.format(userID, user_full.shape[0]))
    print (user_full['title'].head(5))
    print ('Recommending the highest {0} predicted ratings books not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating book that the user hasn't seen yet.
    recommendations = (books_df[~books_df['book_id'].isin(user_full['book_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'book_id',
               right_on = 'book_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

#    return user_full, recommendations
    return recommendations


In [28]:
recommend_books(preds_df, 229, books_df, ratings_df, 5)

User 229 has already rated 45 books. The first 5 are:
35    The Girl Who Played with Fire (Millennium, #2)
41                               Pride and Prejudice
22     Taran Wanderer (The Chronicles of Prydain #4)
24            The Colossus Rises (Seven Wonders, #1)
28                                 Wuthering Heights
Name: title, dtype: object
Recommending the highest 5 predicted ratings books not already rated.


,book_id,title
2668,13449,"Unbroken: A World War II Story of Survival, Re..."
1550,10962,"The Lost Symbol (Robert Langdon, #3)"
3758,188,Jane Eyre
543,13456,"A Discovery of Witches (All Souls Trilogy, #1)"
1459,9881,The Last Olympian (Percy Jackson and the Olymp...
